In [111]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

In [112]:
df_model = pd.read_csv("files/input/results_fifa_ranking.csv")
df_model = df_model[df_model["home_score"] < 5]
df_model

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_win,home_team_strength,away_team_strength,home_team_form_goals,home_team_form_points,away_team_form_goals,away_team_form_points,home_advantage
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,draw,1497.46,1794.90,0.0,0,0.0,0,1
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,True,1794.90,1497.46,0.0,1,0.0,1,1
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,True,1497.46,1794.90,0.4,1,0.8,4,1
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,draw,1794.90,1497.46,1.0,4,0.8,4,1
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,True,1497.46,1794.90,1.2,5,1.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47070,2024-03-26,Scotland,Northern Ireland,0.0,1.0,Friendly,Glasgow,Scotland,False,False,1497.46,1341.05,1.2,2,1.2,7,1
47071,2024-03-26,Senegal,Benin,1.0,0.0,Friendly,Amiens,France,True,True,1624.73,1225.68,2.4,13,1.0,3,0
47072,2024-03-26,Slovenia,Portugal,2.0,0.0,Friendly,Ljubljana,Slovenia,False,True,1427.84,1748.11,1.4,10,3.4,15,1
47073,2024-03-26,Spain,Brazil,3.0,3.0,Friendly,Madrid,Spain,False,draw,1727.50,1788.65,1.8,12,0.6,4,1


In [113]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
import statsmodels.api as sm

# Assume df_model is already defined and loaded with data

# Define features and target variable
features = ['home_team_strength', 'away_team_strength', 'home_team_form_goals', 'home_team_form_points', 'away_team_form_goals', 'away_team_form_points', 'home_advantage']
# If additional features are to be included based on 'home_team_' or 'away_team_', they should be added here
# features.extend([col for col in df_model.columns if 'home_team_' in col or 'away_team_' in col])

# Drop rows with missing values
df_model = df_model.dropna(how="any")

# Convert features to numeric
X = df_model[features].apply(pd.to_numeric, errors='raise')

# Convert target variable to int
y = df_model["home_score"].astype(int)  # 1 for home win, 0 for away win or draw

# Add constant to X
X = sm.add_constant(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to numpy arrays and ensure correct dtype
X_train = np.asarray(X_train).astype(np.float64)
X_test = np.asarray(X_test).astype(np.float64)
y_train = np.asarray(y_train).astype(np.float64)
y_test = np.asarray(y_test).astype(np.float64)

# Train the model
log_reg = sm.OLS(y_train, X_train)
model = log_reg.fit()

# Make predictions on the test set
y_pred = model.predict(X_test)
df = pd.DataFrame(data=y_pred,columns=["y_pred"])
df['rounded_scores'] = df['y_pred'].apply(lambda x: round(x, 0))

# Access the rounded scores
rounded_scores = df['rounded_scores']

# Calculate evaluation metrics
accuracy = mean_squared_error(y_test, rounded_scores)

accuracy

1.3331007535584705

In [114]:
def rsquared_robust(y, y_pred, weights):
    numerator = np.sum(weights * (y - y_pred) ** 2)
    yw_bar = np.sum(weights * y)  / np.sum(weights)
    denominator = np.sum(weights * (y -yw_bar) ** 2)
    r2_robust = 1 - (numerator / denominator)
    return r2_robust

def adj_r2_robust(x,y,r_squared):
    n = len(x)
    q = x.shape[1]
    if n-q <= 0:
        return pd.NA
    
    r2_adj_robust = 1-(1-r_squared) * ((n-1) / (n-q))

    return r2_adj_robust

In [115]:
display(y)

0        0
1        4
2        2
3        2
4        3
        ..
47070    0
47071    1
47072    2
47073    3
47074    2
Name: home_score, Length: 35828, dtype: int32

In [116]:
from statsmodels.robust.norms import HuberT
import math
model = sm.RLM(y, X, M=HuberT())
result = model.fit()

y_pred = result.predict(X)

rsquard = rsquared_robust(y.squeeze(), y_pred, result.weights)
adj_r2 = adj_r2_robust(X,y,rsquard)
condition_nr = np.linalg.cond(X.mul(np.sqrt(result.weights), axis=0))
jb, jb_pv, skew, kurtosis = sm.stats.jarque_bera(result.resid, axis=0)


In [117]:
display(rsquard,adj_r2, condition_nr, jb,jb_pv,skew,kurtosis,result.summary())

0.1110963664327439

0.1109226555049111

12204.264730028373

1629.9658202391058

0.0

0.4840953443332647

2.606976670415397

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:             home_score   No. Observations:                35828
Model:                            RLM   Df Residuals:                    35820
Method:                          IRLS   Df Model:                            7
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 21 Jun 2024                                         
Time:                        20:02:51                                         
No. Iterations:                    18                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.2140      0.038     31.736      0.000       1.139       1.289
home_team_strength        0.0008   2.53e-05     32.860      0.000       0.001       0.001
away_team_strength       -0.0008   2.49e-05    -32.960      0.000      -0.001      -0.001
home_team_form_goals      0.1548      0.010     15.263      0.000       0.135       0.175
home_team_form_points     0.0101      0.002      4.186      0.000       0.005       0.015
away_team_form_goals      0.0705      0.010      7.149      0.000       0.051       0.090
away_team_form_points    -0.0552      0.002    -22.683      0.000      -0.060      -0.050
home_advantage            0.1671      0.014     11.606      0.000       0.139       0.195
=========================================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""